In [1]:
!nvidia-smi

Sun Feb 13 11:42:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !pip install wandb
# !pip install opencv-python==4.1.2.30  # ==3.4.8.29 ==4.1.2.30 
# !pip install albumentations==0.5.2
# !pip install timm

In [4]:
import os
import random
import math

from glob import glob
from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torch.nn.modules.loss import _Loss
from torch.optim.lr_scheduler import _LRScheduler

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import timm

In [5]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

# Dataset

In [6]:
class MyDataset(Dataset):
    def __init__(self, data, transforms, color, root='./train/train'):
        self.data = data
        self.transforms = transforms
        self.root = root
        self.color = color
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        src, trg = self.data[idx]
        img = cv2.imread(self.root + '/' + src)
        if self.color == 'gray':
            img = np.mean(img, axis=-1)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img, trg
    

In [7]:
from albumentations.augmentations.transforms import GaussNoise
def get_train_transforms():
    return A.Compose([
        A.HorizontalFlip(),
        # A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1), # fold0 동일
        # A.VerticalFlip(), # fold0 동일
        # A.OneOf([
        #          A.MedianBlur(blur_limit=5),
        #          A.MotionBlur(blur_limit=5),
        #          A.GaussianBlur(blur_limit=5)
        # ]), # fold0 동일
        A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=10), # fold0 동일 fold1 오름 
        # A.GaussNoise(var_limit=10),
        A.Normalize(mean=[0.59534838, 0.5949003 , 0.59472117], std=[0.29705278, 0.29707744, 0.29702731]),
        ToTensorV2()
    ])

def get_valid_transforms():
    return A.Compose([
        A.Normalize(mean=[0.59534838, 0.5949003 , 0.59472117], std=[0.29705278, 0.29707744, 0.29702731]),
        ToTensorV2()
    ])

In [8]:
def split_stratified_shuffle_split(fold, n_split, seed):
    skf = StratifiedShuffleSplit(n_splits=n_split, train_size=0.8, test_size=0.2, random_state=seed)
    for idx, i in enumerate(skf.split(df['file_name'], df['COVID'])):
        if idx == fold:
            train_data = df.values[i[0]]

            valid_data = df.values[i[1]]
    return train_data, valid_data

# Model

In [9]:
timm.list_models('*')

# m = timm.create_model('seresnet18', pretrained=False)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_tiny_hnf',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_x

In [10]:
# m.fc.in_features

In [11]:
class MyModel(nn.Module):
    def __init__(self, n_classes, model_name):
        super(MyModel, self).__init__()
        self.feature = timm.create_model(model_name, pretrained=False)
        # convnext 계열
        # self.feature.head.fc = nn.Linear(in_features=2048, out_features=n_classes, bias=True) 

        # resnet 계열
        self.out_features = self.feature.fc.in_features
        self.feature.fc = nn.Linear(in_features=self.out_features, out_features=self.out_features//4, bias=True) 
        self.out = nn.Linear(in_features=self.out_features//4, out_features=n_classes, bias=True)
        self.drop = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.feature(x)
        x = self.drop(x)
        x = self.out(x)
        return x

# Initialization

In [12]:
def init_weight(model, kind='xavier'):
    for name, i in model.named_parameters():
        if kind == 'xavier':
            if i.dim() < 2:
                continue
            if 'weight' in name:
                init.xavier_normal_(i, gain=1.0)
            elif 'bias' in name:
                init.xavier_uniform_(i, gain=1.0)
            else:
                pass
        elif kind == 'kaiming':
            if i.dim() < 2:
                continue
            if 'weight' in name:
                init.kaiming_normal_(i)
            elif 'bias' in name:
                init.kaiming_uniform_(i)
            else:
                pass

# Loss

In [13]:
class F1_Loss(nn.Module):
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, 2).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)
        
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        return 1 - f1.mean()

In [14]:
class MyLoss(_Loss):
    def __init__(self): # 실험환경 세팅으로
        super(MyLoss, self).__init__()
        self.lossCE = nn.CrossEntropyLoss()
        self.lossF1 = F1_Loss()
        
    def forward(self, preds, trg):
        return (self.lossCE(preds, trg) + self.lossF1(preds, trg)) / 2

# Train Functions

In [15]:
def train_one_epoch(model, criterion, train_loader, scheduler, device):
    model.train()
    
    losses = 0
    match = 0
    for idx, (src, trg) in tqdm(enumerate(train_loader)):
        src, trg = src.to(device).float(), trg.to(device).long()
        
        outs = model(src)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, trg)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        losses += loss.item()
        match += (preds == trg).sum().item()
    return losses, match
        
def valid_one_epoch(model, criterion, valid_loader, device):
    model.eval()
    print("validation")
    
    losses = 0
    match = 0
    with torch.no_grad():
        for idx, (src, trg) in tqdm(enumerate(valid_loader)):
            src, trg = src.to(device).float(), trg.to(device).long()
            
            outs = model(src)
            preds = torch.argmax(outs, dim=-1)
            
            loss = criterion(outs, trg)
            
            losses += loss.item()
            match += (preds == trg).sum().item()

    return losses, match

# Early Stop

In [16]:
class EarlyStopper():

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.best_acc = 0
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, score: float)-> None:

        if self.best_acc == 0:
            self.best_acc = score
            return None

        elif score <= self.best_acc:
            self.patience_counter += 1
            self.save_model = False
            if self.patience_counter == self.patience:
                self.stop = True
                
        elif score > self.best_acc:
            self.patience_counter = 0
            self.save_model = True
            self.best_acc = score
            print('best score :', self.best_acc)
        print("best_acc", self.best_acc)

# Scheduler

In [17]:
class CosineAnnealingWarmupRestarts(_LRScheduler):
    """
        optimizer (Optimizer): Wrapped optimizer.
        first_cycle_steps (int): First cycle step size.
        cycle_mult(float): Cycle steps magnification. Default: -1.
        max_lr(float): First cycle's max learning rate. Default: 0.1.
        min_lr(float): Min learning rate. Default: 0.001.
        warmup_steps(int): Linear warmup step size. Default: 0.
        gamma(float): Decrease rate of max learning rate by cycle. Default: 1.
        last_epoch (int): The index of last epoch. Default: -1.
    """

    def __init__(self,
                 optimizer: torch.optim.Optimizer,
                 first_cycle_steps: int,
                 cycle_mult: float = 1.,
                 max_lr: float = 0.1,
                 min_lr: float = 0.001,
                 warmup_steps: int = 0,
                 gamma: float = 1.,
                 last_epoch: int = -1
                 ):
        assert warmup_steps < first_cycle_steps

        self.first_cycle_steps = first_cycle_steps  # first cycle step size
        self.cycle_mult = cycle_mult  # cycle steps magnification
        self.base_max_lr = max_lr  # first max learning rate
        self.max_lr = max_lr  # max learning rate in the current cycle
        self.min_lr = min_lr  # min learning rate
        self.warmup_steps = warmup_steps  # warmup step size
        self.gamma = gamma  # decrease rate of max learning rate by cycle

        self.cur_cycle_steps = first_cycle_steps  # first cycle step size
        self.cycle = 0  # cycle count
        self.step_in_cycle = last_epoch  # step size of the current cycle

        super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)

        # set learning rate min_lr
        self.init_lr()

    def init_lr(self):
        self.base_lrs = []
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.min_lr
            self.base_lrs.append(self.min_lr)

    def get_lr(self):
        if self.step_in_cycle == -1:
            return self.base_lrs
        elif self.step_in_cycle < self.warmup_steps:
            return [(self.max_lr - base_lr) * self.step_in_cycle / self.warmup_steps + base_lr for base_lr in
                    self.base_lrs]
        else:
            return [base_lr + (self.max_lr - base_lr) \
                    * (1 + math.cos(math.pi * (self.step_in_cycle - self.warmup_steps) \
                                    / (self.cur_cycle_steps - self.warmup_steps))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.step_in_cycle = self.step_in_cycle + 1
            if self.step_in_cycle >= self.cur_cycle_steps:
                self.cycle += 1
                self.step_in_cycle = self.step_in_cycle - self.cur_cycle_steps
                self.cur_cycle_steps = int(
                    (self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
        else:
            if epoch >= self.first_cycle_steps:
                if self.cycle_mult == 1.:
                    self.step_in_cycle = epoch % self.first_cycle_steps
                    self.cycle = epoch // self.first_cycle_steps
                else:
                    n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                    self.cycle = n
                    self.step_in_cycle = epoch - int(
                        self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                    self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** (n)
            else:
                self.cur_cycle_steps = self.first_cycle_steps
                self.step_in_cycle = epoch

        self.max_lr = self.base_max_lr * (self.gamma ** self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

# Train

In [18]:
# !wandb login

In [19]:
import wandb

In [ ]:
CFG = {
    'seed':2022,
    'fold':2, # 학습시킬 fold
    'n_split':5, # fold 개수
    'batch_size': 16,
    'num_classes': 2,
    'epoch': 200,
    'model_name': "seresnet18", # seresnet18, resnext50_32x4d
    'initialization': "kaiming", # kaiming, xavier
    'color': 'rgb'
}

root_dir = '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification'

wandb.config = CFG
experiment_name = '2Aug' # BrightContrast Vertical
run = wandb.init(project=f"{CFG['model_name']}", settings=wandb.Settings(start_method="thread"), name=f"{experiment_name}_fold{CFG['fold']}")

wandb: Currently logged in as: yeardream7 (use `wandb login --relogin` to force relogin)


In [18]:
# seed 고정
seed_everything(CFG['seed'])
# device 설정
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# Data 설정
df = pd.read_csv(root_dir + '/' + 'train/train.csv')
train_data, valid_data = split_stratified_shuffle_split(CFG['fold'], CFG['n_split'], CFG['seed'])

train_transforms = get_train_transforms() # augmentation
valid_transforms = get_valid_transforms()

train_dataset = MyDataset(train_data, train_transforms, CFG['color'], root_dir + '/' + 'train/train/')
valid_dataset = MyDataset(valid_data, valid_transforms, CFG['color'], root_dir + '/' + 'train/train/')

train_loader = DataLoader(
    train_dataset, 
    batch_size=CFG['batch_size'],
    num_workers=0,
    shuffle=True,
    pin_memory=use_cuda,
    drop_last=True,)

valid_loader = DataLoader(
    valid_dataset, 
    batch_size=CFG['batch_size'],
    num_workers=0,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False)
# 모델, Loss, scheduler, optimizer 설정
model = MyModel(CFG['num_classes'], CFG['model_name']).to(device)
init_weight(model, kind=CFG['initialization'])

criterion = MyLoss().to(device)

cosine_annealing_scheduler_arg = dict(
    first_cycle_steps=len(train_dataset)//CFG['batch_size'] * CFG['epoch'],
    cycle_mult=1.0,
    max_lr=1e-04,
    min_lr=1e-07,
    warmup_steps=len(train_dataset)//CFG['batch_size'] * 3,
    gamma=0.9
)

optimizer = optim.Adam(model.parameters(), lr=0.000, weight_decay=0)
scheduler = CosineAnnealingWarmupRestarts(optimizer, **cosine_annealing_scheduler_arg)

early_stopper = EarlyStopper(patience=100)

In [19]:
print('Start Training!')
for i in range(CFG['epoch']):
    print(f"Epoch :", i)
    train_losses, train_match = train_one_epoch(model, criterion, train_loader, scheduler, device)
    print("Train loss, score:", train_losses / len(train_loader), train_match / len(train_loader.dataset))
    valid_losses, valid_match = valid_one_epoch(model, criterion, valid_loader, device)
    print("Valid loss, score:", valid_losses / len(valid_loader), valid_match / len(valid_loader.dataset))
    early_stopper.check_early_stopping(valid_match / len(valid_loader.dataset))

    wandb_dict = {
        'train loss': train_losses / len(train_loader),
        'train score': train_match / len(train_loader.dataset),
        'valid loss': valid_losses / len(valid_loader),
        'valid score': valid_match / len(valid_loader.dataset),
        'learning rate': scheduler.get_lr()[0]
    }

    wandb.log(wandb_dict)

    print(scheduler.get_lr())

    if early_stopper.save_model == True:
        dic = {
            'model':model.state_dict(),
            'optimizer':optimizer.state_dict(),
            'scheduler':scheduler.state_dict(),
        }
        torch.save(dic, root_dir + f"/pth/{CFG['fold']}_best_{CFG['model_name']}.pth")
        print('save_model')

    if early_stopper.stop:
        break

os.rename(
    root_dir + f"/pth/{CFG['fold']}_best_{CFG['model_name']}.pth", 
    root_dir + f"/pth/{CFG['fold']}_{early_stopper.best_acc:.4f}_{CFG['model_name']}_{experiment_name}.pth"
    )

Start Training!
Epoch : 0


0it [00:00, ?it/s]

Train loss, score: 0.6488315919414163 0.5058139534883721
validation


0it [00:00, ?it/s]

Valid loss, score: 0.573972139093611 0.6538461538461539
[3.3400000000000005e-05]
Epoch : 1


...

Valid loss, score: 0.17938919291676333 0.9076923076923077
best_acc 0.9230769230769231
[1.0635131402778672e-07]
Epoch : 199


0it [00:00, ?it/s]

Train loss, score: 0.07525514044391457 0.9593023255813954
validation


0it [00:00, ?it/s]

Valid loss, score: 0.18019010129177737 0.8923076923076924
best_acc 0.9230769230769231
[1e-07]


# Ensemble

In [23]:
model_paths = [
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/0_0.9231_seresnet18_Compose200.pth',
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/1_0.8846_seresnet18_Compose200.pth',
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/2_0.8692_seresnet18_Compose200.pth',
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/3_0.9385_seresnet18_Compose200.pth',
    '/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/pth/4_0.9385_seresnet18_Compose200.pth'
]

In [24]:
model = MyModel(CFG['num_classes'], CFG['model_name']).to(device)
df_test = pd.read_csv(root_dir + '/' + 'test/sample_submission.csv')

test_transforms = get_valid_transforms()
test_dataset = MyDataset(df_test.values, test_transforms, CFG['color'], root_dir + '/' + 'test/test/')

test_loader = DataLoader(
    test_dataset, 
    batch_size=CFG['batch_size'],
    num_workers=0,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False)

In [25]:
result = None

with torch.no_grad():
    for i in tqdm(model_paths):

        checkpoint = torch.load(i)
        model.load_state_dict(checkpoint['model'])
        model.eval()

        for j in range(2): 
            tmp = []
            for idx, (src, trg) in tqdm(enumerate(test_loader)):
                
                src, trg = src.to(device).float(), trg.to(device).long()
                if j == 0:
                    src = torch.flip(src, [-1])
                outs = model(src)
                tmp.extend(outs.detach().cpu().numpy().tolist())
            if result is None:
                result = np.array(tmp)
            else:
                result += np.array(tmp)

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
preds = np.argmax(result, axis=-1) # base tta
preds 

array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0])

In [ ]:
preds = np.argmax(result, axis=-1) 
preds

array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0])

In [26]:
preds = np.argmax(result, axis=-1) #
preds

array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0])

In [27]:
df_test['COVID'] = preds

In [28]:
df_test.to_csv('/content/drive/Othercomputers/내 컴퓨터/workspace/dl_classification/submission/submission_seresnet18_Compose.csv', index=False)

# GradCAM 

In [ ]:
!pip install grad-cam

In [ ]:
model

In [ ]:
gradcam_loader = DataLoader(
    valid_dataset, 
    batch_size=1,
    num_workers=0,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False)

(384, 384, 3)

In [ ]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

model.cpu()
img = cv2.imread(root_dir + '/train/train' + '/119.png')

# model = resnet50(pretrained=True)
target_layers = [model.feature.layer4[-1]]
input_tensor = img # Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers, use_cuda=args.use_cuda) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.
# targets = [ClassifierOutputTarget(281)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
# grayscale_cam = cam(input_tensor=input_tensor, target_category=target_category)
grayscale_cam = cam(input_tensor=input_tensor)
# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)

Exception: ignored